<h4>本章重点介绍Spark在集群上执行代码的全过程</h4>

- Spark应用程序的体系结构和组件
- Spark应用程序的生命周期
- 重要的底层执行属性，例如流水线处理
- 运行一个Spark应用程序都需要什么

<h4>Spark应用程序的体系结构</h4>

- 首先细说下Spark应用程序模块

<h4>Spark驱动器</h4>

- Spark驱动器程序控制应用程序的进程，负责整个应用程序的执行并且维护着Spark集群的状态，即执行器的任务和状态
- Spark驱动器必须与集群管理器交互才能获得物理资源，并启动执行器
- 简而言之，Spark驱动器只是物理机器上的一个进程，负责维护集群上运行的应用程序状态

<h4>Spark执行器</h4>

- Spark执行器也是一个进程，负责执行由Spark驱动器分配的任务
- Spark执行器核心功能：运行驱动器分配的任务、报告成功或失败的状态和执行结果
- 每个Spark应用程序都有自己的执行器进程

<h4>集群管理器</h4>

- Spark驱动器和执行器依靠集群管理器联系在一起
- 集群管理器有自己的集群驱动器和集群工作节点的抽象
- 与Spark执行器、驱动器的区别在于集群管理器管理的<b>是物理机器而非进程</b>
- 实际运行Spark应用程序时，我们从集群管理器中请求运行Spark驱动器的机器资源和Spark执行器的计算资源

<h4>Spark目前支持三个集群管理器</h4>

- 内置独立集群管理器
- Apache Mesos
- Hadoop YARN

- 在运行应用程序之前，需要选择执行模式，以确定计算资源的物理位置，常用的有以下三种
    - 集群模式
    - 客户端模式
    - 本地模式

<h4>集群模式</h4>

- 将预编译的JAR包、Python脚本提交给集群管理器
- 除了执行器进程外，集群管理器会在集群内的某个工作节点上启动驱动器进程
- 这意味着集群管理器负责维护所有与Spark应用程序相关的进程

<h4>客户端模式</h4>

- 与集群模式几乎相同，只是Spark驱动器保留在提交应用程序的客户端机器中
- 这些机器通常被称为网关机器（gateway machines）和边缘节点（edge nodes）

<h4>本地模式</h4>

- 在一台机器上运行整个Spark应用程序
- 通过单机的线程实现并行性

<h4>Spark应用程序的生命周期（Spark外部）</h4>

- 以下介绍Spark应用程序从初始化到退出的生命周期
- 假设，我们使用集群模式，即由集群工作节点启动Spark驱动器和执行器
- 假设该集群运行了四个节点，其中包括一个驱动节点和三个工作节点

1. 客户请求

- 第一步是提交应用程序，应用程序通常是编译好的JAR包或者库
- 首先向集群管理器的驱动节点发起请求，为Spark驱动器进程显式地请求资源
- 集群管理器接受请求并将驱动器放到集群中的一个物理节点上
- 之后提交应用程序的客户端退出，应用程序开始在集群上运行
- 以下为需要运行的命令

./bin/spark-submit \
    --class \<main-class> \\<br>
    --master \<master-url> \\<br>
    --deploy-mode cluster \\<br>
    --conf \<key>=\<value> 

2. 启动

- 现在驱动器已经被放到集群上了，它开始执行用户代码
- 此代码必须包含一个初始化Spark集群（如驱动器和若干执行器）的SparkSession
- SparkSession随后将于集群管理器的驱动节点通信，要求它在集群上启动Spark执行器
- 集群管理器随后在集群工作节点上启动执行器
- 执行器的数量及相关配置由用户通过最开始spark-submit调用中的命令行参数设置


- 假如一切顺利，集群管理器会启动Spark执行器，并将程序执行位置等相关信息发送给Spark驱动器
- 所有程序正确关联后，Spark集群构建完成

3. 执行

- 自此，集群的驱动节点和工作节点相互通信、执行代码和移动数据
- 驱动节点将任务安排到每个工作节点上
- 每个工作节点回应给驱动节点这些任务的执行状态，回复启动成功或启动失败

4. 完成

- Spark应用程序完成后，Spark驱动器会以成功或失败状态退出
- 集群管理器会为该驱动器关闭集群中的执行器
- 可以向集群管理器询问，Spark应用程序是成功退出还是失败退出

<h4>Spark应用程序的生命周期（Spark内部）</h4>

- 每个应用程序由一个或多个Spark作业组成，这一系列作业是串行执行的
- 除非使用多线程并行启动多个作业

<h4>SparkSession</h4>

- 任何Spark应用程序第一步都是创建一个SparkSession，交互模式中通常预先创建
- 但在应用程序中需要自己创建
- 以下是SparkSession的创建方法，构建器方法，该方法可以稳定实例化Spark和SQL Context
- 创建SparkSession后，就可以访问和运行Spark代码

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Word Count")\
    .config("spark.some.config.option", "some-value")\
    .getOrCreate()

<h4>SparkContext</h4>

- SparkSession中的SparkContext对象代表与Spark集群的连接
- 可以通过它与一些Spark的低级API进行通信
- 在早期的示例文档中，通常以变量sc存储
- 通过SparkContext，可以创建RDD、累加器和广播变量
- 如今的SparkSession = SparkContext + SQLContext
- 需要注意的是，你应该永远不需要使用SQLContext，并尽量避免使用SparkContext

In [ ]:
spark.sparkContext.parallelize(range(1, 21))

<h4>逻辑指令</h4>

- Spark代码基本上由转换和动作组成
- 以下首先使用一个简单的DataFrame执行三步
    1. 重新分区
    2. 执行逐个值的操作
    3. 执行聚合操作并收集最终结果

In [2]:
df1 = spark.range(2, 10000000, 2)
df2 = spark.range(2, 10000000, 4)
step1 = df1.repartition(5)
step12 = df2.repartition(6)
step2 = step1.selectExpr("id * 5 as id")
step3 = step2.join(step12, ["id"])
step4 = step3.selectExpr("sum(id)")

In [27]:
df2.rdd.getNumPartitions()

1

- 当你调用collect或任何动作时，你将执行Spark作业
- Spark作业由阶段和任务组成
- 如果正在本地计算机上运行以查看Spark UI，请在浏览器上访问localhost: 4040

In [23]:
step4.explain()

== Physical Plan ==
*(7) HashAggregate(keys=[], functions=[sum(id#8L)])
+- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#417]
   +- *(6) HashAggregate(keys=[], functions=[partial_sum(id#8L)])
      +- *(6) Project [id#8L]
         +- *(6) SortMergeJoin [id#8L], [id#2L], Inner
            :- *(3) Sort [id#8L ASC NULLS FIRST], false, 0
            :  +- Exchange hashpartitioning(id#8L, 200), ENSURE_REQUIREMENTS, [id=#401]
            :     +- *(2) Project [(id#0L * 5) AS id#8L]
            :        +- Exchange RoundRobinPartitioning(5), REPARTITION_WITH_NUM, [id=#397]
            :           +- *(1) Range (2, 10000000, step=2, splits=1)
            +- *(5) Sort [id#2L ASC NULLS FIRST], false, 0
               +- Exchange hashpartitioning(id#2L, 200), ENSURE_REQUIREMENTS, [id=#408]
                  +- Exchange RoundRobinPartitioning(6), REPARTITION_WITH_NUM, [id=#407]
                     +- *(4) Range (2, 10000000, step=4, splits=1)




<h4>Spark作业</h4>

- 一个动作触发一个Spark作业
- 每个作业可以分为一系列阶段，引擎在shuffle操作之后会启动新的阶段
- 一个阶段可以分为多个任务

<h4>阶段（stage）</h4>

- Spark会将作业内部尽可能多的转换操作加入同一个阶段
- 引擎在shuffle操作之后将启动新的阶段
- 一次shuffle操作意味着对数据的物理重分区
- 例如对DataFrame进行排序，或者按key进行分组
- 这种重分区需要检查整个数据集，需要跨执行器的协调来移动数据
- Spark在每次shuffle后都会开始一个新阶段
- 最终按照顺序执行各阶段以计算最终结果

<h4>阶段任务解析</h4>

- 上述例子总共有六个阶段
    - 第一二阶段
        - spark执行range操作，将创建DataFrame，默认使用8个分区，也就各有8个任务
    - 第三四阶段
        - 通过shuffle操作改变分区数量，DataFrame被shuffle成5个分区和6个分区，各有5个和6个任务
    - 第五阶段
        - 第五阶段是连接操作，有200个任务，因为spark.sql.shuffle.partitions的默认值是200
        - 这也意味着在执行过程中执行一个shuffle操作时，默认输出200个shuffle分区
    - 第六阶段
        - 执行sum聚合操作

- 以下是改变Spark SQL默认分区配置的方法

In [28]:
spark.conf.set("spark.sql.shuffle.partitions", 50)

<h4>任务（task）</h4>

- Spark中的阶段由若干任务（task）组成，每个任务都对应于一组数据和一组将在单个执行器上运行的转换操作
- 如果数据集只有一个大分区，那我们只有一个任务
- 如果有1000个小分区，就有1000个可以并行执行的任务
- 任务是应用于每个数据单元的计算单位，将数据划分为更多单位意味着可以并行执行更多分区

- 分区数量的经验法则是分区数量应该大于集群执行器数量
- 如果在本地计算机上运行代码，应该将分区数设得较低，因为本地计算机不太可能并行执行任务
- 无论分区数量设置为多少，整个阶段都是并行执行的

<h4>Spark执行的关键优化——流水线执行</h4>

- Spark关键优化，它在RDD级别或其以下级别上执行
- 如果一系列转换操作不需要任何跨节点得数据移动，就可以将这些操作合并为一个单独的任务阶段
- 例如如果编写一个基于RDD的编程，首先执行map操作，然后filter，然后map
    - 然而这些操作不需要在节点间移动数据，那么就可以将它们合并为单一阶段的任务
    - 这会比每步完成后将中间结果写入内存或磁盘要快得多

<h4>shuffle数据持久化</h4>

- 当Spark需要运行跨节点移动数据的操作时，例如按键约减的操作（reduce-by-key）
- 处理引擎执行跨网络的shuffle操作
- 在Spark执行shuffle操作时，总是让前一阶段的源任务将要发送的数据写入到本地磁盘的shuffle文件
- 之后的按键分组和约减就从这个shuffle文件中获取数据